<a href="https://colab.research.google.com/github/arbhavana/NLP-With-Disaster-Tweets/blob/main/Code%20file/NLP_with_disaster_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing the neccessary libraries and downloading the required packages.

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

# Download NLTK data files
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Importing the **Training Dataset** into **data** and **Testing dataset** into **test_data**

In [ ]:
# Load the datasets
train_file = 'final_combined_file.csv'
test_file = 'test.csv'
data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)

In [ ]:
data.head()

,id,keyword,location,text,target
0,0.0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,1.0,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1
2,2.0,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1
3,3.0,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1
4,4.0,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0


In [ ]:
count = data['target'].value_counts()
print(count)

target
0    9256
1    2315
Name: count, dtype: int64


### Data cleaning and the conversion into tokens

In [ ]:
# Preprocess the data
def preprocess_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@w+|\#','', text)  # Remove mentions and hashtags
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z]', ' ', text)  # Remove punctuation and numbers
    tokens = nltk.word_tokenize(text)  # Tokenize
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Remove stopwords
    return tokens

data['cleaned_text'] = data['text'].apply(preprocess_text)
test_data['cleaned_text'] = test_data['text'].apply(preprocess_text)

# Drop the columns if they exist, handling potential KeyError
columns_to_drop = ['id', 'location', 'keyword']
for col in columns_to_drop:
    if col in data.columns:
        data.drop(col, axis=1, inplace=True)

print(data.columns)


Index(['text', 'target', 'cleaned_text'], dtype='object')


Creating the word embeddings for the tokens and saving the embedding techniques models in the pickle files.


In [ ]:
# Word2Vec embeddings
sentences = data['cleaned_text'].tolist() + test_data['cleaned_text'].tolist()
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
word2vec_model.train(sentences, total_examples=len(sentences), epochs=10)

def get_word2vec_embeddings(tokens):
    valid_tokens = [word for word in tokens if word in word2vec_model.wv]
    if valid_tokens:
        embedding = np.mean([word2vec_model.wv[word] for word in valid_tokens], axis=0)
    else:
        embedding = np.zeros(100)
    return embedding

data['embeddings'] = data['cleaned_text'].apply(get_word2vec_embeddings)
test_data['embeddings'] = test_data['cleaned_text'].apply(get_word2vec_embeddings)

X = np.vstack(data['embeddings'].values)
  # Assuming 'target' is the column name for labels

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['target'])

word2vec_model_file = 'word2vec_model.pkl'
word2vec_model.save(word2vec_model_file)

label_encoder_file = 'label_encoder.pkl'
joblib.dump(label_encoder, label_encoder_file)

['label_encoder.pkl']

### Training the model on Random Forest, SVM and Logistic Regression algorithms.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Assuming the data is preprocessed and embeddings are generated as shown earlier

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'SVM': SVC(kernel='linear', random_state=42)
}

accuracies = {}
# Train and evaluate each model
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)

    accuracy = accuracy_score(y_val, y_val_pred)
    accuracies[model_name] = accuracy


    print(f"{model_name} Classification Report (Validation):")
    print(classification_report(y_val, y_val_pred))
    print(f"{model_name} Accuracy (Validation): {accuracy_score(y_val, y_val_pred)}\n")

 # Find the model with the highest accuracy
best_model_name = max(accuracies, key=accuracies.get)
best_accuracy = accuracies[best_model_name]

print(f"The model with the highest accuracy is: {best_model_name} with an accuracy of {best_accuracy}")




Random Forest Classification Report (Validation):
              precision    recall  f1-score   support

           0       0.89      0.96      0.92      1866
           1       0.75      0.51      0.61       449

    accuracy                           0.87      2315
   macro avg       0.82      0.73      0.76      2315
weighted avg       0.86      0.87      0.86      2315

Random Forest Accuracy (Validation): 0.8721382289416847

Logistic Regression Classification Report (Validation):
              precision    recall  f1-score   support

           0       0.87      0.95      0.91      1866
           1       0.68      0.41      0.51       449

    accuracy                           0.85      2315
   macro avg       0.78      0.68      0.71      2315
weighted avg       0.83      0.85      0.83      2315

Logistic Regression Accuracy (Validation): 0.8483801295896328

SVM Classification Report (Validation):
              precision    recall  f1-score   support

           0       0.86  

### Hyperparameter tuning using RandomizedSearchCV for the random forest algorithm.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define a smaller parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'sqrt']
}

# Initialize the random forest classifier
rf = RandomForestClassifier(random_state=42)

# Initialize RandomizedSearchCV with fewer iterations
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=20, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Perform random search
rf_random.fit(X_train, y_train)

# Print the best parameters found
print("Best Parameters:")
print(rf_random.best_params_)

# Use the best estimator to predict on validation set
best_rf = rf_random.best_estimator_
y_val_pred_tuned = best_rf.predict(X_val)

# Print classification metrics after tuning
print("Classification Report after Tuning (Validation):")
print(classification_report(y_val, y_val_pred_tuned))
print(f"Accuracy after Tuning (Validation): {accuracy_score(y_val, y_val_pred_tuned)}")

# Save the best model
best_model_file = 'best_model_tuned.pkl'
joblib.dump(best_rf, best_model_file)


Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
39 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
39 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_

Best Parameters:
{'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20}
Classification Report after Tuning (Validation):
              precision    recall  f1-score   support

           0       0.89      0.96      0.92      1866
           1       0.75      0.53      0.62       449

    accuracy                           0.87      2315
   macro avg       0.82      0.74      0.77      2315
weighted avg       0.87      0.87      0.87      2315

Accuracy after Tuning (Validation): 0.873866090712743


['best_model_tuned.pkl']

### Making predictions and the test data and saving the predictions in predictions.csv

In [ ]:


# Load the trained models and other necessary objects
best_model_file = 'best_model_tuned.pkl'
word2vec_model_file = 'word2vec_model.pkl'
label_encoder_file = 'label_encoder.pkl'

best_rf = joblib.load(best_model_file)
word2vec_model = Word2Vec.load(word2vec_model_file)
label_encoder = joblib.load(label_encoder_file)


# Prepare the data for prediction
X_test = np.vstack(test_data['embeddings'].values)

# Predict the sentiments
y_test_pred = best_rf.predict(X_test)
y_test_pred_labels = label_encoder.inverse_transform(y_test_pred)

# Add predictions to the test data
test_data['prediction'] = y_test_pred_labels

# Save the predictions to a new file
predictions_file = 'predictions.csv'
test_data[['text', 'prediction']].to_csv(predictions_file, index=False)

print(f"Predictions saved to {predictions_file}")


Predictions saved to predictions.csv


### Testing the model with single sentence depicting a disaster

In [ ]:
# Test the model with a single sentence
def test_single_sentence(sentence):
    tokens = preprocess_text(sentence)
    embedding = get_word2vec_embeddings(tokens)
    embedding = embedding.reshape(1, -1)
    prediction = best_rf.predict(embedding)
    return prediction

# Example test with the provided test data
test_sentences = [

    "the Tsunami killed thousands.",
    "the thunderstorm hit our neighbour's house",
    "a house flew in the air due to whirlwind this afternoon",
    "Massive fire has broken out near the oak street bridge in richmond of british colombia",
    "Firefighters are working tirelessly to contain a fire at a local business. Support them by avoiding the area.",
    "Severe thunderstorm warning in effect. Stay indoors and avoid travel if possible."
    "Recovery and rebuilding after the tsunami will require a global effort. Let's stand together.",
    "Tornado spotted! Take cover immediately and follow emergency instructions.",
    "Hurricane making landfall with strong winds and heavy rain. Evacuate if advised and stay indoors.",
    "A strong earthquake has just struck. Check on your loved ones and follow safety protocols.",
    "Smoky skies and blazing fires. Our thoughts are with everyone in the path of the wildfires.",
    "Inundated streets and rising waters. Thoughts are with everyone affected by the floods.",
    "Downed trees and power lines reported due to the storm. Stay away from any fallen wires.",

]

for sentence in test_sentences:
    prediction = test_single_sentence(sentence)
    print(f"Prediction for the test sentence '{sentence}': {prediction}")

Prediction for the test sentence 'the Tsunami killed thousands.': [1]
Prediction for the test sentence 'the thunderstorm hit our neighbour's house': [1]
Prediction for the test sentence 'a house flew in the air due to whirlwind this afternoon': [1]
Prediction for the test sentence 'Massive fire has broken out near the oak street bridge in richmond of british colombia': [1]
Prediction for the test sentence 'Firefighters are working tirelessly to contain a fire at a local business. Support them by avoiding the area.': [0]
Prediction for the test sentence 'Severe thunderstorm warning in effect. Stay indoors and avoid travel if possible.Recovery and rebuilding after the tsunami will require a global effort. Let's stand together.': [1]
Prediction for the test sentence 'Tornado spotted! Take cover immediately and follow emergency instructions.': [1]
Prediction for the test sentence 'Hurricane making landfall with strong winds and heavy rain. Evacuate if advised and stay indoors.': [1]
Predic

### Testing the model with single sentence which are not disaster related

In [ ]:
# Test the model with a single sentence
def test_single_sentence(sentence):
    tokens = preprocess_text(sentence)
    embedding = get_word2vec_embeddings(tokens)
    embedding = embedding.reshape(1, -1)
    prediction = best_rf.predict(embedding)
    return prediction

# Example test with the provided test data
test_sentences = [
    "Since it's yoga day, it's the perfect image to signify that our infrastructure is stretching itself as far towardsthe skies as possible ",

    "The first train to cross the world's highest railway bridge-the Chenab bridge in india",
    "Just had the best ice cream sundae ever!",
    "Spending the afternoon at the park with my dog.",
    "Started a new series on Netflix and I'm hooked.",
    "Had a productive day at work today.",
    "Went for a morning run and feel great!",
    "Cooking a new recipe for dinner tonight.",
    "Just booked tickets for a vacation next month!",
    "Enjoying a peaceful evening with a good book.",
    "Went shopping today and found some great deals.",
    "Attending a friend’s wedding this weekend.",
    "Spent the day at the museum, so much fun!",
    "Just got a promotion at work, feeling accomplished!",
    "Decorating the house for the holidays.",
    "Visited a new coffee shop and loved it.",
    "Had a relaxing spa day, feeling refreshed.",
    "Took a scenic drive through the countryside.",
    "Just finished a challenging puzzle, feeling proud!",
    "Had a game night with friends, it was a blast!",
    "Trying out yoga for the first time today.",
    "Enjoying some homemade cookies with a cup of tea.",
    "Had a fun day at the amusement park.",
    "Learning a new language, it's so interesting!",
    "Went to a farmer's market and bought fresh produce.",
    "Spending quality time with loved ones this weekend.",
    "Attended a fascinating lecture on astronomy.",
    "Just planted a new garden in the backyard.",
    "Exploring new hiking trails this summer.",
    "Enjoying a lazy Sunday afternoon at home.",
    "Volunteering at the local animal shelter today.",
    "Binge-watching my favorite TV series.",
    "Just got a new haircut, feeling great!",
    "Had an amazing dinner date last night.",
    "Trying out new baking recipes in the kitchen.",
    "Went fishing for the first time and caught a big one!",
    "Taking up painting as a new hobby.",
    "Just finished reading an inspiring book.",
    "Spent the day at the beach, soaking up the sun.",
    "Playing board games with the family tonight.",
    "Excited to attend a music festival next month!"
]

for sentence in test_sentences:
    prediction = test_single_sentence(sentence)
    print(f"Prediction for the test sentence '{sentence}': {prediction}")


Prediction for the test sentence 'Since it's yoga day, it's the perfect image to signify that our infrastructure is stretching itself as far towardsthe skies as possible ': [0]
Prediction for the test sentence 'The first train to cross the world's highest railway bridge-the Chenab bridge in india': [0]
Prediction for the test sentence 'Just had the best ice cream sundae ever!': [0]
Prediction for the test sentence 'Spending the afternoon at the park with my dog.': [0]
Prediction for the test sentence 'Started a new series on Netflix and I'm hooked.': [0]
Prediction for the test sentence 'Had a productive day at work today.': [0]
Prediction for the test sentence 'Went for a morning run and feel great!': [0]
Prediction for the test sentence 'Cooking a new recipe for dinner tonight.': [0]
Prediction for the test sentence 'Just booked tickets for a vacation next month!': [0]
Prediction for the test sentence 'Enjoying a peaceful evening with a good book.': [0]
Prediction for the test senten

In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install joblib
!pip install numpy
!pip install nltk
!pip install gensim
!pip install requests
!pip install base64

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.0 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement base64 (from versions: none)
ERROR: No matching distribution found for base64


In [ ]:
%%writefile app.py
import streamlit as st
import joblib
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import requests
import base64
from datetime import datetime, timedelta

# Download NLTK data files
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load the trained models and other necessary objects
model = joblib.load('best_model_tuned.pkl')
word2vec_model = Word2Vec.load('word2vec_model.pkl')
label_encoder = joblib.load('label_encoder.pkl')

# Load and encode the background image
def get_base64_image(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode()
    return encoded_string

#bg_image = get_base64_image('earth-planet.webp')  # Replace with actual image path

# Custom CSS
st.markdown(
    f"""
    <style>


    .stButton>button {{
        background-color: #071952;
        color: white;
        padding: 10px 24px;
        font-size: 16px;
        border: none;
        border-radius: 5px;
    }}
    .stTextInput>div>div>input {{
        padding: 5px;
        font-size: 16px;
    }}
    .disaster {{
        color: white;
        background-color: red;
        padding: 10px;
        border-radius: 5px;
    }}
    .non-disaster {{
        color: white;
        background-color: green;
        padding: 10px;
        border-radius: 5px;
    }}

    </style>
    """,
    unsafe_allow_html=True
)

# Preprocess the input text
def preprocess_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#','', text)  # Remove mentions and hashtags
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z]', ' ', text)  # Remove punctuation and numbers
    tokens = nltk.word_tokenize(text)  # Tokenize
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Remove stopwords
    return tokens

def get_word2vec_embeddings(tokens):
    valid_tokens = [word for word in tokens if word in word2vec_model.wv]
    if valid_tokens:
        embedding = np.mean([word2vec_model.wv[word] for word in valid_tokens], axis=0)
    else:
        embedding = np.zeros(100)
    return embedding

# Function to fetch tweet text from URL
def fetch_tweet_text_from_url(url, bearer_token):
    tweet_id = url.split('/')[-1]
    tweet_url = f"https://api.twitter.com/2/tweets/{tweet_id}"
    headers = {
        'Authorization': f'Bearer {bearer_token}',
    }
    response = requests.get(tweet_url, headers=headers)
    if response.status_code != 200:
        st.error(f"Error fetching content from URL: {response.status_code}")
        st.write(response.json())  # Debugging step to show the response content
        return None
    tweet = response.json()
    return tweet['data']['text']

# Function to fetch disaster tweets


def fetch_disaster_tweets(timeframe, bearer_token):
    query = "disaster OR flood OR earthquake OR hurricane OR tornado OR tsunami OR wildfire OR landslide OR eruption OR cyclone OR drought OR blizzard OR storm OR catastrophe OR calamity OR emergency OR plane crash -is:retweet"

    end_time = datetime.utcnow() - timedelta(seconds=10)  # Ensure end_time is at least 10 seconds before current time
    start_time = end_time - timedelta(hours=4 if timeframe == "Last 4 hours" else 24)
    start_time_str = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_time_str = end_time.strftime("%Y-%m-%dT%H:%M:%SZ")

    search_url = f"https://api.twitter.com/2/tweets/search/recent?query={query}&start_time={start_time_str}&end_time={end_time_str}&max_results=10"
    headers = {
        'Authorization': f'Bearer {bearer_token}',
    }

    response = requests.get(search_url, headers=headers)
    if response.status_code != 200:
        st.error(f"Error fetching tweets: {response.status_code}")
        st.write(response.json())  # Debugging step to show the response content
        return []
    tweets = response.json().get('data', [])
    users = {user['id']: user['location'] for user in response.json().get('includes', {}).get('users', [])}


    disaster_tweets = []
    for tweet in tweets:
        tweet_text = tweet['text']
        cleaned_text = preprocess_text(tweet_text)
        embedding = get_word2vec_embeddings(cleaned_text).reshape(1, -1)
        prediction = model.predict(embedding)
        prediction_label = label_encoder.inverse_transform(prediction)[0]
        if prediction_label == 1:  # Assuming 1 represents disaster
            user_id = tweet['author_id']
            user_location = users.get(tweet['author_id'], 'Unknown location')
            disaster_tweets.append({'text': tweet_text, 'location': user_location})

    return disaster_tweets


# Streamlit app
def main():
    st.markdown("<div class='main'>", unsafe_allow_html=True)
    st.title("Disaster Tweet Classifier")
    st.write("Enter a tweet or a URL leading to a tweet to predict if it's a disaster tweet or not.")
    tweet_or_url = st.text_area("Enter Tweet Text or URL")

    if st.button("Fetch & Classify"):
        if tweet_or_url.startswith("http"):
            tweet_text = fetch_tweet_text_from_url(tweet_or_url, 'insert_your_bearer_token_here')  # Replace with your actual bearer token
            if tweet_text:
                st.write(f"Tweet: {tweet_text}")
                cleaned_text = preprocess_text(tweet_text)
            else:
                st.error("Failed to fetch content from URL.")
                return
        else:
            tweet_text = tweet_or_url
            cleaned_text = preprocess_text(tweet_or_url)

        embedding = get_word2vec_embeddings(cleaned_text).reshape(1, -1)
        prediction = model.predict(embedding)
        prediction_label = label_encoder.inverse_transform(prediction)[0]

        if prediction_label == 1:
            st.markdown(f"<h3 class='disaster'>Prediction: Disaster Tweet</h3>", unsafe_allow_html=True)
        else:
            st.markdown(f"<h3 class='non-disaster'>Prediction: Not a Disaster Tweet</h3>", unsafe_allow_html=True)

    st.write("## Fetch Recent Disaster Tweets")
    timeframe = st.selectbox("Select timeframe", ["Last 24 hours", "Last 4 hours"])
    if st.button("Fetch Disaster Tweets"):
        tweets = fetch_disaster_tweets(timeframe, 'Replace with your actual bearer token')  # Replace with your actual bearer token
        if tweets:
            for tweet in tweets:
                st.write(f"- {tweet['text']}")
        else:
            st.write("No recent disaster tweets found.")

    st.markdown("</div>", unsafe_allow_html=True)

if __name__ == '__main__':
    main()


Overwriting app.py


In [ ]:
from pyngrok import ngrok
import subprocess
import time

# Set up your ngrok authentication token
ngrok.set_auth_token('Replace with your actual ngrok auth token')  # Replace with your actual ngrok auth token

# Kill any existing ngrok tunnels
ngrok.kill()

# Run the Streamlit app
streamlit_proc = subprocess.Popen(['streamlit', 'run', 'app.py', '--server.port', '8501'])

# Wait a few seconds for the app to start
time.sleep(20)  # Increase the sleep time if necessary

# Create a tunnel to the Streamlit port 8501
public_url = ngrok.connect(8501)
print('Public URL:', public_url)

# Keep the tunnel open
try:
    streamlit_proc.communicate()
except KeyboardInterrupt:
    streamlit_proc.terminate()
    ngrok.kill()


Public URL: NgrokTunnel: "https://a676-34-139-251-178.ngrok-free.app" -> "http://localhost:8501"
